# 1. DATA PRE-PROCESSING

In [1]:
import pandas as pd 
import numpy as np 

In [2]:
df_rain = pd.read_csv('../data/rainfall.csv')
df_temp = pd.read_csv('../data/temp.csv')
df_pest = pd.read_csv('../data/pesticides.csv')
df_yield = pd.read_csv('../data/yield.csv')

## Converting Datatypes

In [3]:
df_rain.dtypes

 Area                            object
Year                              int64
average_rain_fall_mm_per_year    object
dtype: object

In [4]:
df_temp.dtypes

year          int64
country      object
avg_temp    float64
dtype: object

In [5]:
df_pest.dtypes

Domain      object
Area        object
Element     object
Item        object
Year         int64
Unit        object
Value      float64
dtype: object

In [6]:
df_yield.dtypes

Domain Code     object
Domain          object
Area Code        int64
Area            object
Element Code     int64
Element         object
Item Code        int64
Item            object
Year Code        int64
Year             int64
Unit            object
Value            int64
dtype: object

In [7]:
# we're converting 'average_rain_fall_mm_per_year' to float type

In [8]:
df_rain['average_rain_fall_mm_per_year'].unique()

array(['327', '1485', '89', nan, '1010', '1030', '591', '562', '534',
       '1110', '447', '..', '1292', '83', '2666', '1422', '618', '847',
       '1705', '1039', '2200', '1146', '1028', '416', '1761', '2722',
       '608', '748', '1274', '228', '1904', '1604', '537', '1342', '322',
       '1522', '645', '3240', '900', '1543', '1646', '2926', '1348',
       '1113', '1335', '498', '677', '703', '220', '2083', '1410', '2274',
       '51', '1784', '2156', '383', '626', '788', '848', '2592', '536',
       '867', '1831', '836', '1026', '700', '1187', '652', '2350', '1996',
       '1651', '1577', '2387', '1440', '1976', '589', '1940', '1083',
       '2702', '216', '1118', '435', '832', '2051', '1668', '111', '250',
       '630', '1054', '121', '533', '1834', '641', '661', '2391', '56',
       '656', '934', '619', '1513', '1181', '2875', '1972', '282', '560',
       '92', '2041', '758', '450', '241', '346', '1032', '2091', '285',
       '1500', '778', '1732', '2280', '151', '1150', '1414', 

In [9]:
df_rain['average_rain_fall_mm_per_year'] = pd.to_numeric(
    df_rain['average_rain_fall_mm_per_year'], 
    errors='coerce'
)

In [10]:
df_rain.dtypes

 Area                             object
Year                               int64
average_rain_fall_mm_per_year    float64
dtype: object

## Normalizing column names

In [11]:
df_rain.columns = df_rain.columns.str.strip().str.lower().str.replace(" ", "_")
df_temp.columns = df_temp.columns.str.strip().str.lower().str.replace(" ", "_")
df_pest.columns = df_pest.columns.str.strip().str.lower().str.replace(" ", "_")
df_yield.columns = df_yield.columns.str.strip().str.lower().str.replace(" ", "_")

## Standardizing features and feature names + Dropping irrelevant columns 

In [12]:
df_rain.columns, df_temp.columns, df_pest.columns, df_yield.columns

(Index(['area', 'year', 'average_rain_fall_mm_per_year'], dtype='object'),
 Index(['year', 'country', 'avg_temp'], dtype='object'),
 Index(['domain', 'area', 'element', 'item', 'year', 'unit', 'value'], dtype='object'),
 Index(['domain_code', 'domain', 'area_code', 'area', 'element_code', 'element',
        'item_code', 'item', 'year_code', 'year', 'unit', 'value'],
       dtype='object'))

In [13]:
df_rain=df_rain.rename(columns={'area':'country'})
df_pest=df_pest.rename(columns={'area':'country'})
df_yield=df_yield.rename(columns={'area':'country', 'item':'crop'})

df_yield.drop(['domain_code', 'domain', 'area_code', 'element_code', 'element', 'item_code', 'year_code'], axis = 1, inplace = True)
df_pest.drop(['domain', 'element', 'item'], axis = 1, inplace = True)

In [14]:
df_pest.drop(['unit'], axis = 1 , inplace = True)
df_pest.rename(columns={'value': 'pesticide_tonnes'}, inplace = True)

In [15]:
df_yield.drop(['unit'], axis = 1 , inplace = True)
df_yield.rename(columns={'value': 'yield_hg_ha'}, inplace = True)

**Converting object types to lowercase**

In [16]:
df_rain.country = df_rain.country.str.lower()
df_temp.country = df_temp.country.str.lower()
df_pest.country = df_pest.country.str.lower()
df_yield.country = df_yield.country.str.lower()


df_yield.crop = df_yield.crop.str.lower()

## Filter for relevant field values

In [17]:
# we're only going to look at data after 1990

df_rain = df_rain[df_rain['year'] >= 1990]
df_temp = df_temp[df_temp['year'] >= 1990]
df_pest = df_pest[df_pest['year'] >= 1990]
df_yield = df_yield[df_yield['year'] >= 1990]



## Handling missing, duplicate and invalid values

In [18]:
from IPython.display import display

In [19]:
display(df_rain.isna().sum())
display(df_temp.isna().sum())
display(df_pest.isna().sum())
display(df_yield.isna().sum())

country                            0
year                               0
average_rain_fall_mm_per_year    677
dtype: int64

year        0
country     0
avg_temp    0
dtype: int64

country             0
year                0
pesticide_tonnes    0
dtype: int64

country        0
crop           0
year           0
yield_hg_ha    0
dtype: int64

**no missing values anymore**

In [20]:
display(df_rain.duplicated().sum())
display(df_temp.duplicated().sum())
display(df_pest.duplicated().sum())
display(df_yield.duplicated().sum())

np.int64(0)

np.int64(723)

np.int64(0)

np.int64(0)

**these duplicates correspond to different states/regions within the country - so we keep them for now**

### Checking for Outliers

In [21]:
# USING IQR

def iqr(df, feature):
    Q1 = df[feature].quantile(0.25)
    Q3 = df[feature].quantile(0.75)
    IQR = Q3 - Q1
    
    outliers = df[(df[feature] < Q1 - 1.5*IQR) | (df[feature] > Q3 + 1.5*IQR)]
    print(f"Method: IQR | Feature: {feature}")
    display(outliers)


# USING MODIFIED Z-SCORE

def modified_z_score(df, feature):
    median = np.median(df[feature])
    MAD = np.median(np.abs(df[feature] - median))
    modified_z = 0.6745 * (df[feature] - median) / MAD
    outliers = df[np.abs(modified_z) > 3.5]
    print(f"Method: Modified Z-Score | Feature: {feature}")
    display(outliers) 

In [22]:
# df_rain

iqr(df_rain,'average_rain_fall_mm_per_year')
modified_z_score(df_rain,'average_rain_fall_mm_per_year')

Method: IQR | Feature: average_rain_fall_mm_per_year


,country,year,average_rain_fall_mm_per_year


Method: Modified Z-Score | Feature: average_rain_fall_mm_per_year


,country,year,average_rain_fall_mm_per_year


### Categorical Filtering / Normalization

**Removing territories, dependencies, or small regions that not fully recognized as sovereign countries in the ISO sense from `country`**

In [23]:
non_countries = [
    "American Samoa", "Aruba", "Bahamas", "Bermuda", "British Virgin Islands",
    "Cayman Islands", "Channel Islands", "Curacao", "Faroe Islands",
    "French Polynesia", "Gibraltar", "Greenland", "Guam", "Hong Kong SAR, China",
    "Isle of Man", "Kosovo", "Macao SAR, China", "Monaco", "New Caledonia",
    "Northern Mariana Islands", "San Marino", "Sint Maarten (Dutch part)",
    "St. Martin (French part)", "Tonga", "Turks and Caicos Islands",
    "Virgin Islands (U.S.)"
]

df_rain = df_rain[~df_rain['country'].isin(non_countries)]
df_temp = df_temp[~df_temp['country'].isin(non_countries)]
df_pest = df_pest[~df_pest['country'].isin(non_countries)]
df_yield = df_yield[~df_yield['country'].isin(non_countries)]